<a href="https://colab.research.google.com/github/aliabdi064/AI-Powered-Invoice-Automation/blob/main/VATOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This cell connects Colab to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile

# --- CONFIGURATION ---
# 1. PASTE THE PATH to your zip file here
zip_file_path = '/content/drive/My Drive/10invoices.zip'

# 2. DEFINE a directory to store the extracted PDFs
output_extract_directory = '/content/invoices/'
# ---

# Create the output directory if it doesn't exist
if not os.path.exists(output_extract_directory):
    os.makedirs(output_extract_directory)

# Unzip the file
print(f"Extracting {zip_file_path}...")
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_extract_directory)

print("Extraction complete.")
print(f"{len(os.listdir(output_extract_directory))} files extracted.")


Extracting /content/drive/My Drive/10invoices.zip...
Extraction complete.
19 files extracted.


In [ ]:
!pip install -q -U google-generativeai
!pip install pdf2image
!pip install gspread oauth2client pandas
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 0s (1,179 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126319 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import google.generativeai as genai
from google.colab import auth, userdata
import gspread
from google.auth import default
import pandas as pd

# --- AUTHENTICATION ---

# Part 1: Gemini API
# Use Colab's built-in secret manager for your API key.
# Click the "Key" icon on the left sidebar, add a new secret,
# name it "GEMINI_API_KEY", and paste your key as the value.
try:
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print("Gemini API Key configured successfully.")
except userdata.SecretNotFoundError:
    print("ERROR: Secret 'GEMINI_API_KEY' not found. Please follow the instructions to add it.")
except Exception as e:
    print(f"An error occurred during Gemini configuration: {e}")


# Part 2: Google Sheets
# This authenticates your user account to allow writing to Sheets.
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("Google Sheets authentication successful.")
except Exception as e:
    print(f"An error occurred during Google Sheets authentication: {e}")

Gemini API Key configured successfully.
Google Sheets authentication successful.


In [ ]:
import json
import time
from pdf2image import convert_from_path

# --- SCRIPT CONFIGURATION ---
# 1. The name of the Google Sheet you created
sheet_name = 'Invoice Data Output'

# 2. Directory where the PDFs were extracted
invoices_dir = '/content/invoices/'
# ---

# Initialize the Gemini Model
model = genai.GenerativeModel('gemini-1.5-flash')

# The magic prompt for Gemini
prompt_template = """
You are an expert invoice data extraction AI. Your task is to analyze the image(s) from an invoice and extract the following information.
The entire invoice may be spread across one or more images.
Provide the output ONLY in a valid JSON format. Do not include any text before or after the JSON object.

The required fields are:
- "customer_name": The name of the customer or company being invoiced.
- "invoice_number": The unique identifier for the invoice.
- "invoice_date": The date the invoice was issued.
- "po_number": The Purchase Order number, if available. If not found, use "N/A".
- "line_items": An array of objects, where each object represents a line item with "description", "quantity", "unit_price", and "line_total_before_vat".
- "subtotal": The total value of all items before VAT or taxes.
- "vat_amount": The total VAT or tax amount. If not specified, calculate it from the total and subtotal.
- "invoice_total": The final, total amount of the invoice including all taxes and charges.

Please analyze the following invoice image(s) and return the extracted data in the specified JSON format.
"""

# Open the Google Sheet and select the first worksheet
try:
    sh = gc.open(sheet_name)
    worksheet = sh.get_worksheet(0)
    print(f"Successfully opened Google Sheet: '{sheet_name}'")

    # --- NEW: Robust Header Writing Logic ---
    # Define the exact header row we expect
    header = [
        "File Name", "Customer Name", "Invoice Number", "Invoice Date",
        "PO Number", "Subtotal", "VAT Amount", "Invoice Total", "Line Item Details", "AI Raw Output"
    ]

    # Get the first row of the sheet to check if the header is already there
    first_row = []
    try:
        first_row = worksheet.row_values(1)
    except gspread.exceptions.APIError as e:
        # This error can occur if the sheet is completely empty, which is fine.
        if "exceeds grid limits" not in str(e):
            raise e

    # If the first row is not our header, insert the header at the top
    if first_row != header:
        worksheet.insert_row(header, 1)
        print("Wrote header row to sheet.")
    else:
        print("Header row already present.")


except gspread.exceptions.SpreadsheetNotFound:
    print(f"ERROR: Spreadsheet '{sheet_name}' not found. Please check the name and sharing settings.")
except Exception as e:
    print(f"An error occurred opening the sheet: {e}")


# --- MAIN PROCESSING LOOP ---
pdf_files = [f for f in os.listdir(invoices_dir) if f.lower().endswith('.pdf')]
print(f"\nFound {len(pdf_files)} PDF files to process. Starting...")

# Start appending data from the next available row after the header
start_row = len(worksheet.get_all_values()) + 1

for filename in pdf_files:
    pdf_path = os.path.join(invoices_dir, filename)
    print(f"\n--- Processing: {filename} ---")

    # 1. Convert PDF to a list of images
    try:
        pil_images = convert_from_path(pdf_path)
    except Exception as e:
        print(f"ERROR: Could not convert PDF to image: {e}")
        worksheet.append_row([filename, f"ERROR: Failed to convert PDF - {e}", "", "", "", "", "", "", "", ""])
        continue

    if not pil_images:
        print("Skipping - PDF conversion resulted in no images.")
        worksheet.append_row([filename, "ERROR: PDF conversion yielded no images.", "", "", "", "", "", "", "", ""])
        continue

    # 2. Prepare content and call the AI
    vision_prompt = [prompt_template] + pil_images
    try:
        response = model.generate_content(vision_prompt)
        ai_response_text = response.text

        # Clean the response to ensure it's valid JSON
        json_str = ai_response_text.strip().replace('```json', '').replace('```', '').strip()

        # 3. Parse the JSON response
        try:
            data = json.loads(json_str)
            line_items_str = json.dumps(data.get('line_items', []))

            # 4. Append data to the Google Sheet
            row_to_add = [
                filename,
                data.get('customer_name', 'N/A'),
                data.get('invoice_number', 'N/A'),
                data.get('invoice_date', 'N/A'),
                data.get('po_number', 'N/A'),
                data.get('subtotal', 'N/A'),
                data.get('vat_amount', 'N/A'),
                data.get('invoice_total', 'N/A'),
                line_items_str,
                json_str
            ]
            worksheet.append_row(row_to_add)
            print(f"SUCCESS: Extracted data for {filename} and added to sheet.")

        except json.JSONDecodeError:
            print(f"ERROR: AI returned malformed JSON. Writing raw output to sheet.")
            worksheet.append_row([filename, "ERROR: Malformed JSON from AI.", "", "", "", "", "", "", "", ai_response_text])

    except Exception as e:
        print(f"ERROR: Failed to call Gemini API for {filename}: {e}")
        worksheet.append_row([filename, f"ERROR: API Call Failed - {e}", "", "", "", "", "", "", "", ""])

    # Be a good citizen and avoid hitting API rate limits
    time.sleep(2) # Wait 2 seconds between requests

print("\n--- All invoices processed! ---")

Successfully opened Google Sheet: 'Invoice Data Output'
Wrote header row to sheet.

Found 19 PDF files to process. Starting...

--- Processing: PO No. 13894 Invoice No. 7300622935 Bond Street Fire.pdf ---
SUCCESS: Extracted data for PO No. 13894 Invoice No. 7300622935 Bond Street Fire.pdf and added to sheet.

--- Processing: PO No. 14275 Invoice No. 7301391704.pdf ---
SUCCESS: Extracted data for PO No. 14275 Invoice No. 7301391704.pdf and added to sheet.

--- Processing: PO No. 14275 Invoice No. 7301567090.pdf ---
SUCCESS: Extracted data for PO No. 14275 Invoice No. 7301567090.pdf and added to sheet.

--- Processing: PO No. 13894 Invoice No. 7301392476.pdf ---
SUCCESS: Extracted data for PO No. 13894 Invoice No. 7301392476.pdf and added to sheet.

--- Processing: PO No. 14275 Invoice No. 7301295083.pdf ---
SUCCESS: Extracted data for PO No. 14275 Invoice No. 7301295083.pdf and added to sheet.

--- Processing: PO No. 13894 Invoice No. 7301567978.pdf ---
SUCCESS: Extracted data for PO No